In this notebook I create a small catalog from the GLADE to the stellar mass estimation.

The code was written by Mária Pálfi (marika97@caesar.elte.hu).

In [1]:
# importing useful packages
import pandas as pd
import numpy as np

## Reading the GLADE catalog

Here I used the redshift in the heliocentric frame.

In [2]:
# reading the catalog to 'data' pandas dataframe
data = pd.read_csv( 'GLADE+2.txt', delimiter = ' ',
                   usecols=[2,3,4,5,7,8,9,19,20,23,25,27,28], # necessary columns  
                   header=None, # there are no header in the txt
                   names=[ "GWGC","Hyp", "2MASS", "wiseX", "type", "ra", "dec",
                          "W1","W2", "z", "flag", "z_err", "lumdis"], # I name the columns
                   low_memory = False )
print( 'dataframe with the necessary columns:\n' )
data

dataframe with the necessary columns:



,GWGC,Hyp,2MASS,wiseX,type,ra,dec,W1,W2,z,flag,z_err,lumdis
0,NGC4736,NGC4736,12505314+4107125,J125053.14+410712.7,G,192.721451,41.120152,5.611000,6.120000,0.000991,0.0,0.000029,4.392418
1,NGC4548,NGC4548,12352642+1429467,J123526.45+142946.9,G,188.860123,14.496320,9.416000,9.306000,0.004120,1.0,0.000119,15.876007
2,NGC6503,NGC6503,17492651+7008396,J174926.45+700840.8,G,267.360474,70.144341,10.180000,10.102000,0.001000,1.0,0.000029,12.446600
3,NGC4442,NGC4442,12280389+0948130,J122803.90+094813.3,G,187.016220,9.803620,8.476000,8.504000,0.003557,1.0,0.000103,11.461371
4,NGC4469,NGC4469,NaN,J122928.05+084500.8,G,187.367000,8.749890,9.784000,9.706000,0.004139,1.0,0.000120,15.180920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23181753,NaN,NaN,NaN,NaN,Q,359.999118,28.954734,17.728901,16.535843,2.452000,0.0,0.113916,20403.464630
23181754,NaN,NaN,NaN,NaN,Q,359.999303,34.720842,16.933657,15.875454,3.109000,0.0,0.135597,27158.368406
23181755,NaN,NaN,NaN,NaN,Q,359.999615,3.268586,14.833991,13.702355,1.232962,0.0,0.073688,8792.439985
23181756,NaN,NaN,NaN,NaN,Q,359.999759,20.721079,16.979166,16.091722,2.009865,0.0,0.099325,16022.500633


## Adding the WISE magnitude errors 

from wiseScosSvm Table (http://ssa.roe.ac.uk/www/ssa_browser.html)

Check *wise_mag_err.ipynb*.

In [3]:
werr_frame = pd.read_csv( 'wise_err.txt', delimiter = '\t' )
werr_frame['wiseX'] = [ i.strip() for i in werr_frame.wiseX.values ]
data = pd.merge( data, werr_frame, on=['wiseX'], how='left' )
print( 'merged dataframe:\n' )
data.head()

merged dataframe:



,GWGC,Hyp,2MASS,wiseX,type,ra,dec,W1,W2,z,flag,z_err,lumdis,w1mCorr,w1mCorrErr,w2mCorr,w2mCorrErr,fromAllSky
0,NGC4736,NGC4736,12505314+4107125,J125053.14+410712.7,G,192.721451,41.120152,5.611,6.120,0.000991,0.0,0.000029,4.392418,NaN,NaN,NaN,NaN,NaN
1,NGC4548,NGC4548,12352642+1429467,J123526.45+142946.9,G,188.860123,14.496320,9.416,9.306,0.004120,1.0,0.000119,15.876007,NaN,NaN,NaN,NaN,NaN
2,NGC6503,NGC6503,17492651+7008396,J174926.45+700840.8,G,267.360474,70.144341,10.180,10.102,0.001000,1.0,0.000029,12.446600,NaN,NaN,NaN,NaN,NaN
3,NGC4442,NGC4442,12280389+0948130,J122803.90+094813.3,G,187.016220,9.803620,8.476,8.504,0.003557,1.0,0.000103,11.461371,NaN,NaN,NaN,NaN,NaN
4,NGC4469,NGC4469,NaN,J122928.05+084500.8,G,187.367000,8.749890,9.784,9.706,0.004139,1.0,0.000120,15.180920,NaN,NaN,NaN,NaN,NaN


In [4]:
len(data[~np.isnan(data.w1mCorrErr)])

20484063

### Adding luminosity distance error

Calculated by Gergely Dálya.

In [5]:
lum_dis_err = pd.read_csv( 'dist_errs.txt', delimiter = ' ', header = None, 
                          names = [ 'lumdis', 'lumdis_err'] )
data['lumdis_err'] = lum_dis_err.lumdis_err
print( 'The dataframe:')
data.head()

The dataframe:


,GWGC,Hyp,2MASS,wiseX,type,ra,dec,W1,W2,z,flag,z_err,lumdis,w1mCorr,w1mCorrErr,w2mCorr,w2mCorrErr,fromAllSky,lumdis_err
0,NGC4736,NGC4736,12505314+4107125,J125053.14+410712.7,G,192.721451,41.120152,5.611,6.120,0.000991,0.0,0.000029,4.392418,NaN,NaN,NaN,NaN,NaN,0.127479
1,NGC4548,NGC4548,12352642+1429467,J123526.45+142946.9,G,188.860123,14.496320,9.416,9.306,0.004120,1.0,0.000119,15.876007,NaN,NaN,NaN,NaN,NaN,0.532731
2,NGC6503,NGC6503,17492651+7008396,J174926.45+700840.8,G,267.360474,70.144341,10.180,10.102,0.001000,1.0,0.000029,12.446600,NaN,NaN,NaN,NaN,NaN,0.128670
3,NGC4442,NGC4442,12280389+0948130,J122803.90+094813.3,G,187.016220,9.803620,8.476,8.504,0.003557,1.0,0.000103,11.461371,NaN,NaN,NaN,NaN,NaN,0.459550
4,NGC4469,NGC4469,NaN,J122928.05+084500.8,G,187.367000,8.749890,9.784,9.706,0.004139,1.0,0.000120,15.180920,NaN,NaN,NaN,NaN,NaN,0.535217


## Writing out the dataframe

In [6]:
data.to_csv( 'data_for_work.txt', sep = '\t', index = False, header = True )